In [43]:
import pandas as pd
import os
import numpy as np
from datetime import datetime
from stat import S_ISREG, ST_CTIME, ST_MODE
import os, sys, time

def read_last_file():
    dir_path = sys.argv[1] if len(sys.argv) == 2 else r'.'
    data = (os.path.join(dir_path, fn) for fn in os.listdir(dir_path))
    data = ((os.stat(path), path) for path in data)

    data = ((stat[ST_CTIME], path)
           for stat, path in data if S_ISREG(stat[ST_MODE]))
    name= []
    for cdate, path in sorted(data):
        name.append(os.path.basename(path))
    return name

def analyze():
    Plac_csv_file_DBM = read_last_file()[-1]
    Plac_csv_file_af = read_last_file()[-2]
    df_DBM = pd.read_csv(Plac_csv_file_DBM)
    df_af = pd.read_csv(Plac_csv_file_af)
    del df_DBM['Advertiser Currency']
    df_af_use = df_af[["Campaign (c)","Installs","new_user_registration (Unique users)","appsflyer_new_order_created (Unique users)"]]
    df_af_use.columns = ["CM Placement ID", "Installs", "Registrations", "Activations" ] 
    df_DBM["CM Placement ID"] = df_DBM["CM Placement ID"].fillna(0).astype(int)
    df_af_use["CM Placement ID"] = df_af_use["CM Placement ID"][df_af_use["CM Placement ID"].apply(lambda x:x.isnumeric()) == True]
    df_af_use["CM Placement ID"] = df_af_use["CM Placement ID"].fillna(0).astype(int)
    pd.merge(df_DBM, df_af_use, on = "CM Placement ID")
    df_af_use = df_af_use.groupby(["CM Placement ID"]).sum()
    df_af_use = df_af_use.reset_index()
    df_col = pd.merge(df_DBM, df_af_use, on = "CM Placement ID")
    df_col.dropna(inplace = True)
    df_col["Insertion Order"] = df_col["Insertion Order"].map({"App Install_Bangalore_Android":"Bangalore", 
                                                               "App Install_Chennai_Android":"Chennai",
                                                               "App Install_Delhi_Android":"Delhi",
                                                              "App Install_Gurgaon_Android": "Gurgaon",
                                                              "App Install_Hyderabad_Android":"Hyderabad",
                                                              "App Install_Pune_Android":"Pune",
                                                              })
    df_col = df_col.groupby(["Insertion Order"]).sum()
    df_col["CTR"] = df_col["Clicks"]/df_col["Impressions"]*100
    df_col["CPI"] = df_col["Revenue (Adv Currency)"]/df_col["Installs"]
    df_col["CPR"] = df_col["Revenue (Adv Currency)"]/df_col["Registrations"]
    df_col["CPA"] = df_col["Revenue (Adv Currency)"]/df_col["Activations"]
    del df_col["CM Placement ID"]
    df_col["CPI"] = df_col.CPI.astype(int)
    df_col["CPR"] = df_col.CPR.astype(int)
    df_col["CPA"] = df_col.CPA.astype(int)
    df_col["CTR"] = df_col.CTR.round(2)
    return df_col
download_name = read_last_file()[-1].split('_')[0]
today = datetime.today()
d3 = today.strftime("%m-%d-%y")
d3 = d3[:5]
file = analyze()
name = download_name + "_" + "City_Analysis_" + d3 + ".xlsx"
file.to_excel(name)

C:\anaconda\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\anaconda\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
